In [17]:
from collections import OrderedDict
import torch
import trimesh

from src.cleansed_cube import SourceCube, make_cube_faces

In [7]:
def make_vert(params):
    return torch.cat([p[0].reshape(3, -1).t()
                      for p in params.values()])


cube = SourceCube(8)
cube

SourceCube()

In [10]:
n, start, end= 8, -0.5, 0.5
        
d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n),
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=0),
    'right': torch.stack(( d1, +d3,  d2), dim=0),    
    'back' : torch.stack((-d3,  d1,  d2), dim=0),         
    'left' : torch.stack(( d1, -d3,  d2), dim=0),
    'top'  : torch.stack(( d1,  d2, +d3), dim=0),
    'down' : torch.stack(( d1,  d2, -d3), dim=0),
})
sides['front'].shape

torch.Size([3, 8, 8])

In [15]:
stacked = torch.stack([p for p in sides.values()])
stacked.shape

torch.Size([6, 3, 8, 8])

In [20]:
faces = make_cube_faces(n)
vertices = stacked.permute(0, 2, 3, 1).reshape(-1, 3)
f = './cube.stl'
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.export(f);

In [32]:
def cube_to_sphere(bxyz):
    x, y, z = bxyz[:, 0, :, :], bxyz[:, 1, :, :], bxyz[:, 2, :, :]
    r = torch.sqrt(x**2 + y**2 + z**2)
    x_angle = torch.atan2(y, x)
    z_angle = torch.acos(z/r)
    
    r = torch.ones_like(x)
    x = r * torch.sin(z_angle) * torch.cos(x_angle)
    y = r * torch.sin(z_angle) * torch.sin(x_angle)
    z = r * torch.cos(z_angle)        
    return torch.stack((x, y, z), dim=1)     


sphered = cube_to_sphere(stacked)
sphered.shape

torch.Size([6, 3, 8, 8])

In [33]:
faces = make_cube_faces(n)
vertices = sphered.permute(0, 2, 3, 1).reshape(-1, 3)
f = './cube_sphere.stl'
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.export(f);

In [ ]:
def cube_to_sphere(bxyz):
    x, y, z = bxyz[:, 0, :, :], bxyz[:, 1, :, :], bxyz[:, 2, :, :]
    r = torch.sqrt(x**2 + y**2 + z**2)
    x_angle = torch.atan2(y, x)
    z_angle = torch.acos(z/r)
    
    r = torch.ones_like(x)
    x = r * torch.sin(z_angle) * torch.cos(x_angle)
    y = r * torch.sin(z_angle) * torch.sin(x_angle)
    z = r * torch.cos(z_angle)        
    return torch.stack((x, y, z), dim=1)   

class SourceCube(nn.Module):
    def __init__(self, n, start=-0.5, end=0.5):
        super(SourceCube, self).__init__()
        d1, d2 = torch.meshgrid(
            torch.linspace(start, end, steps=n),
            torch.linspace(start, end, steps=n))
        d3 = torch.full_like(d1, end) + 1 / n
        sides =  OrderedDict({
            'front': torch.stack((+d3,  d1,  d2), dim=0),
            'right': torch.stack(( d1, +d3,  d2), dim=0),    
            'back' : torch.stack((-d3,  d1,  d2), dim=0),         
            'left' : torch.stack(( d1, -d3,  d2), dim=0),
            'top'  : torch.stack(( d1,  d2, +d3), dim=0),
            'down' : torch.stack(( d1,  d2, -d3), dim=0),
        })
        stacked = torch.stack([p for p in sides.values()])
        sphered = cube_to_sphere(stacked)
        vertices = sphered.permute(0, 2, 3, 1).reshape(-1, 3)
        self.register_buffer('vertices', vertices)
        self.register_buffer('faces', make_cube_faces(n).int())
        self.register_buffer('colors', torch.ones_like(vertices) * 0.5)

    def to_homogeneous(_, t):
        return torch.cat((t,
            torch.ones(t.size(0), 1, device=t.device)), dim=-1)

    def forward(self, deform_verts):
        vert = self.vertices + deform_verts
        pos = self.to_homogeneous(vert)[None]
        return Mesh(pos, self.faces, self.colors)